# Tabnet Image Model

In [1]:
import tabnet
from load_data import *
import os
from tqdm import tqdm_notebook
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
import cv2
tf.compat.v1.enable_eager_execution()


# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

Init Plugin
Init Graph Optimizer
Init Kernel


We only consider a model with the best hyperparameters identified for the individual models

In [2]:
# Load data for best fold of Tabnet
X_train, X_test, X_val, y_train, y_test, y_val = load_data_fold(fold=4, drop_id = False)

## Input process for Image Model

In [3]:
# load paths and labels
img_folder = "data/images_resized"
img_df = pd.read_csv("data/img_paths.csv")
img_df.reset_index(drop=True, inplace = True)

# load label book
label_cat = ["bathroom", "bedroom", "dining", "hallway", "kitchen", "living"]
label = np.arange(6)
label_book = pd.DataFrame({"label": label_cat, "categorical_label": label})

In [4]:
# prices
url_listing = "http://data.insideairbnb.com/ireland/leinster/dublin/2021-11-07/data/listings.csv.gz"
listings = pd.read_csv(url_listing)
urls = listings["listing_url"]
ids = listings["id"]
price = listings["price"]
price = price.str.replace("$","")
price = price.str.replace(",","")
price = price.astype(float)
listings["price"] = price
listings["log_price"] = np.log(price)
listings = listings[listings["price"]<500]


In [5]:
price_df = listings[["log_price", "id"]]
df = pd.merge(img_df, price_df, on = "id", how = "left")
df.head()

,img_path,id,img_no,label,log_price
0,44077_0.png,44077,0,4.0,4.174387
1,44077_1.png,44077,1,5.0,4.174387
2,44077_2.png,44077,2,1.0,4.174387
3,44077_3.png,44077,3,1.0,4.174387
4,44077_4.png,44077,4,1.0,4.174387


In [6]:
filter = np.any(df.isna(), axis = 1)
df = df[~filter]

# drop "others"
filter = df["label"] == 6.0
df = df[~filter]

In [7]:
bool = []
for id in df["id"]:
    tmp = df[df["id"] == id]
    if len(np.unique(tmp["label"])) >= 4:
        bool.append(True)
    else:
        bool.append(False)

In [8]:
df_new = df[bool]
np.unique(df_new["id"].values).shape
df = df_new

In [9]:
df.shape

(56704, 5)

In [10]:
def data_generator(df):
    ''' Extracts the features from a Resnet50 '''
    counter_overall_dummy = 0
    counter_overall_img = 0

    def input_pipeline(room = 0, df = df):
        ids = []
        features = []
        
        # Filter df by room type
        df_room = df[df["label"] == room]
        
        # load resnet
        resnet = tf.keras.applications.resnet.ResNet50(include_top=False, weights='imagenet', pooling="avg", input_shape = (None,None,3))
        resnet_pre = keras.applications.resnet50.preprocess_input
        resnet.trainable = False
        
        # count dummies and images overall
        counter_dummy = 0
        counter_img = 0
        
        # for every id route all images of a room through the resnet50
        for id in tqdm_notebook(np.unique(df["id"])):
            filter = df_room["id"] == id  

            # if an error occurs just add a black images otherwise get resnet features
            try:
                l = []
                
                # if no image of the room is present add a black image and route it throug the resnet
                if filter.sum() == 0:
                    dummy_image = np.zeros((1,256,256,3))
                    dummy_image = resnet_pre(dummy_image)
                    dummy_image = resnet(dummy_image)
                    l.append(dummy_image)
                    counter_dummy += 1             
                
                # load image and get features
                else:
                    path_id = df_room["img_path"][filter]
                    for p in path_id:
                        img_tmp = cv2.imread("data/images_resized/"+p)
                        img_tmp = np.expand_dims(img_tmp, axis = 0)
                        img_tmp = resnet_pre(img_tmp.astype(np.float32))
                        img_tmp = resnet(img_tmp)
                        l.append(img_tmp)
                        counter_img += 1
                        
                # stack all images and compute maximum
                l = np.stack(l)
                l = np.max(l, axis = 0)
                features.append(l)
                ids.append(id)
                
            except:
                dummy_image = np.zeros((1,256,256,3))
                dummy_image = resnet_pre(dummy_image)
                dummy_image = resnet(dummy_image)
                l.append(dummy_image)
                l = np.stack(l)
                l = np.max(l, axis = 0)
                features.append(l)
                ids.append(id)
                continue
            
        # stack all features of the images and leave out zeros features
        features = np.squeeze(np.stack(features))
        filter = np.nonzero(features.sum(axis = 0))[0]
        features = features[:,filter]
        print(len(filter), " features are nonzero.")
        
        features = features.tolist()
        print(counter_dummy, "dummy images were added.")
        
        return features, ids, counter_dummy, counter_img

    basis_df = df[["id","log_price"]]
    basis_df = basis_df.drop_duplicates()
    
    features = []
    ids = []
    
    # loop over all room categories
    for i in tqdm_notebook(np.unique(df["label"])):
        
        feat_cat, ids_cat, counter_dummy, counter_img = input_pipeline(i,df)
        df_tmp = pd.DataFrame({"features_"+str(i): feat_cat, "id": ids_cat})
        basis_df = pd.merge(basis_df, df_tmp, on = "id", how = "left")
        counter_overall_dummy += counter_dummy
        counter_overall_img += counter_img
        
    print(counter_overall_dummy)
    print(counter_overall_img)
    
    return basis_df
    

In [12]:
final_df = data_generator(df)

/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_54342/54665505.py:84: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(np.unique(df["label"])):


  0%|          | 0/6 [00:00<?, ?it/s]

/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_54342/54665505.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id in tqdm_notebook(np.unique(df["id"])):


  0%|          | 0/4356 [00:00<?, ?it/s]

2048  features are nonzero.
266 dummy images were added.


...

  0%|          | 0/4356 [00:00<?, ?it/s]

2048  features are nonzero.
189 dummy images were added.


/var/folders/97/j215pw6x7sq158bvx1ktlhf80000gn/T/ipykernel_54342/54665505.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for id in tqdm_notebook(np.unique(df["id"])):


  0%|          | 0/4356 [00:00<?, ?it/s]

2048  features are nonzero.
342 dummy images were added.
5583
56704


In [13]:
final_df.columns = ["id", "log_price", "bath", "bed", "dining", "hall", "kitchen", "living"]

## Input process for TabNet

In [14]:
# Load data for best fold of Tabnet
X_train, X_test, X_val, y_train, y_test, y_val = load_data_fold(fold=4, drop_id = False)

In [15]:

# set columns for tabnet
bin_col = [col for col in X_train if np.isin(X_train[col].unique(), [0, 1]).all()]
num_col = [col for col in X_train if ~np.isin(X_train[col].unique(), [0, 1]).all()]
col_names = bin_col + num_col
feature_columns = []
col_names.remove("id")
for col in col_names:
    feature_columns.append(tf.feature_column.numeric_column(col))

In [16]:
# merge data for tabnet and images to get consistent ids
data_df_train = pd.merge(final_df, X_train, on="id", how="inner")
data_df_train.dropna(inplace = True)
print("TRAIN SHAPE: ", data_df_train.shape)

data_df_test = pd.merge(final_df, X_test, on="id", how="inner")
data_df_test.dropna(inplace = True)
print("TEST SHAPE: ", data_df_test.shape)

data_df_val = pd.merge(final_df, X_val, on="id", how="inner")
data_df_val.dropna(inplace = True)
print("VAL SHAPE: ", data_df_val.shape)

# extract consistent columns
X_train_tab = data_df_train.filter(col_names)
X_test_tab = data_df_test.filter(col_names)
X_val_tab = data_df_val.filter(col_names)

data_df_train.drop(col_names, axis = 1, inplace = True)
data_df_test.drop(col_names, axis = 1, inplace = True)
data_df_val.drop(col_names, axis = 1, inplace = True)

# extract consistent price
y_train = data_df_train.pop("log_price")
y_test = data_df_test.pop("log_price")
y_val = data_df_val.pop("log_price")

# drop id
X_train = data_df_train.drop("id", axis = 1)
X_test = data_df_test.drop("id", axis = 1)
X_val = data_df_val.drop("id", axis = 1)


TRAIN SHAPE:  (2710, 77)
TEST SHAPE:  (853, 77)
VAL SHAPE:  (706, 77)


In [17]:
def transform(ds):
    # tabnet
    features = tf.unstack(ds["features"])
    features = dict(zip(col_names, features))

    # images
    bath = tf.unstack(ds["bath"])
    bed = tf.unstack(ds["bed"])
    dining = tf.unstack(ds["dining"])
    hall = tf.unstack(ds["hall"])
    kitchen = tf.unstack(ds["kitchen"])
    living = tf.unstack(ds["living"])
    
    prices = ds["price"]
    
    y = prices
    return (bath, bed, dining, hall, kitchen, living, features), y

Construct tensorflow dataset

In [18]:
train_size = int(X_train.shape[0] * 0.9)
batch_size = int(X_train.shape[0] * 0.1)

data_train = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_train["bath"])),
                                                 "bed": np.squeeze(np.stack(X_train["bed"])),
                                                 "dining": np.squeeze(np.stack(X_train["dining"])),
                                                 "hall": np.squeeze(np.stack(X_train["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_train["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_train["living"])),
                                                 "features": X_train_tab,
                                                 "price": y_train})
data_train = data_train.cache()
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(y_train))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(batch_size)

data_test = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_test["bath"])),
                                                 "bed": np.squeeze(np.stack(X_test["bed"])),
                                                 "dining": np.squeeze(np.stack(X_test["dining"])),
                                                 "hall": np.squeeze(np.stack(X_test["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_test["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_test["living"])),
                                                 "features": X_test_tab,
                                                 "price": y_test})
data_test = data_test.cache()
test_dataset = data_test.take(len(y_test))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(batch_size)

data_val = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_val["bath"])),
                                                 "bed": np.squeeze(np.stack(X_val["bed"])),
                                                 "dining": np.squeeze(np.stack(X_val["dining"])),
                                                 "hall": np.squeeze(np.stack(X_val["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_val["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_val["living"])),
                                                 "features": X_val_tab,
                                                 "price": y_val})
data_val = data_val.cache()
val_dataset = data_val.take(len(y_val))
val_dataset = val_dataset.map(transform)
val_dataset = val_dataset.batch(batch_size)

In [19]:
from tensorflow.keras import backend
class weight_constr(tf.keras.constraints.Constraint):
  """Constrains weight tensors to be centered around `ref_value`."""

  def __init__(self):
    self.ref_value = 1

  def __call__(self, w):
    nonneg = w * tf.cast(tf.greater_equal(w, 0.), backend.floatx())
    sum_w = tf.reduce_sum(nonneg)
    nonneg_one = nonneg/sum_w
    return nonneg_one


Model Setup

In [20]:
# setup model
class Compound_model(tf.keras.Model):

  def __init__(self,feature_columns, dropout = 0, l2 = 0, nodes1 = 512, nodes2 = 1,
                 num_features=None,
                 feature_dim=32,
                 output_dim=32,
                 num_decision_steps=3,
                 relaxation_factor=1.5,
                 sparsity_coefficient=1e-5,
                 norm_type='group',
                 batch_momentum=0.98,
                 virtual_batch_size=None,
                 num_groups=1,
                 epsilon=1e-5):
    super().__init__()
    # TabNet
    self.tabnet = tabnet.TabNet(feature_columns=feature_columns,
                          num_features=num_features,
                          feature_dim=feature_dim,
                          output_dim=output_dim,
                          num_decision_steps=num_decision_steps,
                          relaxation_factor=relaxation_factor,
                          sparsity_coefficient=sparsity_coefficient,
                          norm_type=norm_type,
                          batch_momentum=batch_momentum,
                          virtual_batch_size=virtual_batch_size,
                          num_groups=num_groups,
                          epsilon=epsilon)
   
    
    # bathroom
    self.bn1_bath = tf.keras.layers.BatchNormalization()
    self.drop1_bath = tf.keras.layers.Dropout(dropout)
    self.dense1_bath = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_bath = tf.keras.layers.BatchNormalization()
    self.drop2_bath = tf.keras.layers.Dropout(dropout)
    #self.dense2_bath = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_bath = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
 
    # bedroom
    self.bn1_bed = tf.keras.layers.BatchNormalization()
    self.drop1_bed = tf.keras.layers.Dropout(dropout)
    self.dense1_bed = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_bed = tf.keras.layers.BatchNormalization()
    self.drop2_bed = tf.keras.layers.Dropout(dropout)
    #self.dense2_bed = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_bed = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
       
      # dining
    self.bn1_dining = tf.keras.layers.BatchNormalization()
    self.drop1_dining = tf.keras.layers.Dropout(dropout)
    self.dense1_dining = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_dining = tf.keras.layers.BatchNormalization()
    self.drop2_dining = tf.keras.layers.Dropout(dropout)
    #self.dense2_dining = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_dining = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
   
    # hall
    self.bn1_hall = tf.keras.layers.BatchNormalization()
    self.drop1_hall = tf.keras.layers.Dropout(dropout)
    self.dense1_hall = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_hall = tf.keras.layers.BatchNormalization()
    self.drop2_hall = tf.keras.layers.Dropout(dropout)
    #self.dense2_hall = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_hall = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
    
    # kitchen
    self.bn1_kitchen = tf.keras.layers.BatchNormalization()
    self.drop1_kitchen = tf.keras.layers.Dropout(dropout)
    self.dense1_kitchen = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_kitchen = tf.keras.layers.BatchNormalization()
    self.drop2_kitchen = tf.keras.layers.Dropout(dropout)
    #self.dense2_kitchen = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_kitchen = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
  
    # livingroom
    self.bn1_living = tf.keras.layers.BatchNormalization()
    self.drop1_living = tf.keras.layers.Dropout(dropout)
    self.dense1_living = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_living = tf.keras.layers.BatchNormalization()
    self.drop2_living = tf.keras.layers.Dropout(dropout)
    #self.dense2_living = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_living = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
    
    # bring all images together
    self.bn_img = tf.keras.layers.BatchNormalization()
    self.drop_img = tf.keras.layers.Dropout(dropout)
    self.dense_img = tf.keras.layers.Dense(1, activation = tf.nn.relu)
    
    # bring features of TabNet together
    self.dense_tabnet = tf.keras.layers.Dense(1, activation = tf.nn.relu)
        
    # final prediction
    self.bn_final = tf.keras.layers.BatchNormalization()
    self.drop_final = tf.keras.layers.Dropout(dropout)
    self.dense_final = tf.keras.layers.Dense(1, kernel_constraint =weight_constr())
    
  def call(self, inputs, training = None):
    
    # bathroom
    x = self.bn1_bath(inputs[0])
    x = self.drop1_bath(x)
    x = self.dense1_bath(x)
    x = self.bn2_bath(x)
    x = self.drop2_bath(x)
    bath_out = self.dense2_bath(x)
    
    
    # bedroom
    x = self.bn1_bed(inputs[1])
    x = self.drop1_bed(x)
    x = self.dense1_bed(x)
    x = self.bn2_bed(x)
    x = self.drop2_bed(x)
    bed_out = self.dense2_bed(x)
    
    # diningroom
    x = self.bn1_dining(inputs[2])
    x = self.drop1_dining(x)
    x = self.dense1_dining(x)
    x = self.bn2_dining(x)
    x = self.drop2_dining(x)
    dining_out = self.dense2_dining(x)
    
    # hallroom
    x = self.bn1_hall(inputs[3])
    x = self.drop1_hall(x)
    x = self.dense1_hall(x)
    x = self.bn2_hall(x)
    x = self.drop2_hall(x)
    hall_out = self.dense2_hall(x)
    
    # kitchen
    x = self.bn1_kitchen(inputs[4])
    x = self.drop1_kitchen(x)
    x = self.dense1_kitchen(x)
    x = self.bn2_kitchen(x)
    x = self.drop2_kitchen(x)
    kitchen_out = self.dense2_kitchen(x)
    
    # livingroom
    x = self.bn1_living(inputs[5])
    x = self.drop1_living(x)
    x = self.dense1_living(x)
    x = self.bn2_living(x)
    x = self.drop2_living(x)
    living_out = self.dense2_living(x)
    
    
    # tabnet
    self.activations = self.tabnet(inputs[6], training=True)
    out_tabnet = self.dense_tabnet(self.activations)

    # join images
    out_img = tf.keras.layers.concatenate([bath_out, bed_out, dining_out, hall_out, kitchen_out, living_out])
    out_img = self.bn_img(out_img)
    out_img = self.drop_img(out_img)
    out_img = self.dense_img(out_img)

    # join
    out = tf.keras.layers.concatenate([out_img, out_tabnet])
    out = self.bn_final(out)
    out = self.drop_final(out)
    return self.dense_final(out)

In [21]:
def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.math.divide(residual, total))  
  return r2

## Train Compound from Scratch

In [22]:
tf.random.set_seed(123)
model = Compound_model(feature_columns = feature_columns,
                                output_dim=120, feature_dim=125, num_groups=1,
                                num_decision_steps=2, relaxation_factor=1.5,
                                sparsity_coefficient=1e-05, dropout= 0.2, nodes1 = 64, nodes2 = 1, l2 = 0)

logdir = "logs/price/compound/final"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = logdir+"/checkpoint", monitor='val_R_squared', save_best_only=True, mode='max')
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.95, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss='mse' , metrics=[R_squared,"mae"])

model.fit(train_dataset, validation_data = val_dataset, epochs=300, verbose=1)#, callbacks=[checkpoint_callback])

[TabNet]: 5 features will be used for decision steps.


2022-03-01 13:18:25.093681: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:176] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-01 13:18:25.113885: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/300


2022-03-01 13:18:30.070096: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - ETA: 0s - loss: 21.2935 - R_squared: -51.5838 - mae: 4.4726

2022-03-01 13:18:38.168364: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 14s 801ms/step - loss: 21.2935 - R_squared: -51.5838 - mae: 4.4726 - val_loss: 12.9193 - val_R_squared: -29.8014 - val_mae: 3.2501
Epoch 2/300
10/10 [==============================] - 8s 747ms/step - loss: 18.9685 - R_squared: -45.7287 - mae: 4.2737 - val_loss: 6.7390 - val_R_squared: -14.6459 - val_mae: 1.9892
Epoch 3/300
10/10 [==============================] - 8s 791ms/step - loss: 17.2712 - R_squared: -41.3143 - mae: 4.0909 - val_loss: 6.6346 - val_R_squared: -14.2186 - val_mae: 2.2420
Epoch 4/300
10/10 [==============================] - 9s 864ms/step - loss: 15.6217 - R_squared: -37.6281 - mae: 3.8933 - val_loss: 8.2679 - val_R_squared: -18.4434 - val_mae: 2.7211
Epoch 5/300
10/10 [==============================] - 13s 1s/step - loss: 14.0732 - R_squared: -33.6369 - mae: 3.6981 - val_loss: 9.2832 - val_R_squared: -20.8480 - val_mae: 2.9719
Epoch 6/300
10/10 [==============================] - 15s 1s/step - loss: 12.8947 - R_squared: -30.5978

KeyboardInterrupt: 

Evaluate test data

In [47]:
model.evaluate(test_dataset)

4/4 [==============================] - 2s 219ms/step - loss: 0.3427 - R_squared: 0.2178 - mae: 0.4497


[0.3427451252937317, 0.2177862823009491, 0.4497312903404236]

Evaluate weights of final layer

In [48]:
model.layers[-1].get_weights()

[array([[-0.],
        [ 1.]], dtype=float32),
 array([4.179595], dtype=float32)]

# Reload optimal weights from best TabNet and Image Model

In [23]:
model = Compound_model(feature_columns = feature_columns,
                                output_dim=120, feature_dim=125, num_groups=1,
                                num_decision_steps=2, relaxation_factor=1.5,
                                sparsity_coefficient=1e-05, dropout= 0.2, nodes1 = 512, nodes2 = 1, l2 = 0)
lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.95, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss='mse' , metrics=[R_squared,"mae"])

model.fit(train_dataset, validation_data = test_dataset, epochs=2, verbose=1)

[TabNet]: 5 features will be used for decision steps.
Epoch 1/2


2022-03-01 14:18:39.674182: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - ETA: 0s - loss: 21.0671 - R_squared: -50.7218 - mae: 4.4916

2022-03-01 14:18:52.135042: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 22s 1s/step - loss: 21.0671 - R_squared: -50.7218 - mae: 4.4916 - val_loss: 10.3320 - val_R_squared: -26.4928 - val_mae: 2.8735
Epoch 2/2
10/10 [==============================] - 11s 1s/step - loss: 18.9981 - R_squared: -45.5592 - mae: 4.2862 - val_loss: 12.5001 - val_R_squared: -31.6618 - val_mae: 3.3917


## Load best TabNet

In [24]:
X_train_tn, X_test_tn, X_val_tn, y_train_tn, y_test_tn, y_val_tn = load_data(for_dendro = False)

def transform(ds):
    features = tf.unstack(ds["features"])
    prices = ds["price"]

    x = dict(zip(col_names, features))
    y = prices
    return x, y

def R_squared(y, y_pred):
  residual = tf.reduce_sum(tf.square(tf.subtract(y, y_pred)))
  total = tf.reduce_sum(tf.square(tf.subtract(y, tf.reduce_mean(y))))
  r2 = tf.subtract(1.0, tf.math.divide(residual, total))
  
  return r2

bin_col = [col for col in X_train_tn if np.isin(X_train_tn[col].unique(), [0, 1]).all()]
num_col = [col for col in X_train_tn if ~np.isin(X_train_tn[col].unique(), [0, 1]).all()]
col_names = bin_col + num_col

train_size = int(X_train_tn.shape[0] * 0.9)
batch_size = int(X_train_tn.shape[0] * 0.1)

data_train = tf.data.Dataset.from_tensor_slices({"features": X_train_tn, "price": y_train_tn})
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(X_train_tn))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(batch_size)

data_test = tf.data.Dataset.from_tensor_slices({"features": X_val_tn, "price": y_val_tn})
test_dataset = data_test.take(len(X_val_tn))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(batch_size)


feature_columns = []

for col in col_names:
    feature_columns.append(tf.feature_column.numeric_column(col))
od = 120
fd =  125
nds = 2
rf = 1.5
sc = 1e-05

lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.95, staircase=False)
#lr = 0.01
optimizer = tf.keras.optimizers.Adam(lr)

model_tn = tabnet.TabNetRegression(feature_columns, num_regressors=1,
                                output_dim=od, feature_dim=fd, num_groups=1,
                                num_decision_steps=nds, relaxation_factor=rf)

model_tn.compile(optimizer, loss='mse' , metrics=[R_squared, "mse", "mae"])
model_tn.fit(test_dataset)
model_tn.load_weights("TabNet_Selected/best_model3.hdf5")

[TabNet]: 5 features will be used for decision steps.


2022-03-01 14:19:43.549975: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


3/3 [==============================] - 2s 104ms/step - loss: 10.0011 - R_squared: -19.8162 - mse: 10.0011 - mae: 2.6982


## Load best Image Model

In [25]:
def transform(ds):
    bath = tf.unstack(ds["bath"])
    bed = tf.unstack(ds["bed"])
    dining = tf.unstack(ds["dining"])
    hall = tf.unstack(ds["hall"])
    kitchen = tf.unstack(ds["kitchen"])
    living = tf.unstack(ds["living"])
   # others = tf.unstack(ds["others"])

    prices = ds["price"]
    
    y = prices
    return (bath, bed, dining, hall, kitchen, living), y

batch_size = 128

# Load data
X_train, X_test, X_val, y_train, y_test, y_val = load_data_fold(fold=4, drop_id = False)

train_ids = X_train["id"]
test_ids = X_test["id"]
val_ids = X_val["id"]

X_train = pd.merge(train_ids, final_df, on = "id", how = "left")
X_train = X_train.drop(["id","log_price"], axis = 1)
X_train.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]#, "others"]
filter = np.any(X_train.isna(), axis = 1).values
X_train = X_train[~filter]
y_train = y_train[~filter]

X_test = pd.merge(test_ids, final_df, on = "id", how = "left")
X_test = X_test.drop(["id","log_price"], axis = 1)
X_test.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]#, "others"]
filter = np.any(X_test.isna(), axis = 1).values
X_test = X_test[~filter]
y_test = y_test[~filter]

X_val = pd.merge(val_ids, final_df, on = "id", how = "left")
X_val = X_val.drop(["id","log_price"], axis = 1)
X_val.columns = ["bath", "bed", "dining", "hall", "kitchen", "living"]#, "others"]
filter = np.any(X_val.isna(), axis = 1).values
X_val = X_val[~filter]
y_val = y_val[~filter]

data_train = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_train["bath"])),
                                                 "bed": np.squeeze(np.stack(X_train["bed"])),
                                                 "dining": np.squeeze(np.stack(X_train["dining"])),
                                                 "hall": np.squeeze(np.stack(X_train["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_train["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_train["living"])),
                                                 #"others": np.squeeze(np.stack(X_train["others"])),
                                                 "price": y_train})
data_train = data_train.cache()
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(y_train))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(batch_size)

data_test = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_test["bath"])),
                                                 "bed": np.squeeze(np.stack(X_test["bed"])),
                                                 "dining": np.squeeze(np.stack(X_test["dining"])),
                                                 "hall": np.squeeze(np.stack(X_test["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_test["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_test["living"])),
                                                # "others": np.squeeze(np.stack(X_test["others"])),
                                                 "price": y_test})
data_test = data_test.cache()
test_dataset = data_test.take(len(y_test))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(batch_size)

data_val = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_val["bath"])),
                                                 "bed": np.squeeze(np.stack(X_val["bed"])),
                                                 "dining": np.squeeze(np.stack(X_val["dining"])),
                                                 "hall": np.squeeze(np.stack(X_val["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_val["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_val["living"])),
                                                 #"others": np.squeeze(np.stack(X_val["others"])),
                                                 "price": y_val})
data_val = data_val.cache()
val_dataset = data_val.take(len(y_val))
val_dataset = val_dataset.map(transform)
val_dataset = val_dataset.batch(batch_size)

In [26]:
# setup model
class Img_model(tf.keras.Model):

  def __init__(self, dropout = 0, l2 = 0, nodes1 = 512, nodes2 = 1):
    super().__init__()
    
    # bathroom
    self.bn1_bath = tf.keras.layers.BatchNormalization()
    self.drop1_bath = tf.keras.layers.Dropout(dropout)
    self.dense1_bath = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_bath = tf.keras.layers.BatchNormalization()
    self.drop2_bath = tf.keras.layers.Dropout(dropout)
    #self.dense2_bath = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_bath = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
 
    # bedroom
    self.bn1_bed = tf.keras.layers.BatchNormalization()
    self.drop1_bed = tf.keras.layers.Dropout(dropout)
    self.dense1_bed = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_bed = tf.keras.layers.BatchNormalization()
    self.drop2_bed = tf.keras.layers.Dropout(dropout)
    #self.dense2_bed = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_bed = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
       
      # dining
    self.bn1_dining = tf.keras.layers.BatchNormalization()
    self.drop1_dining = tf.keras.layers.Dropout(dropout)
    self.dense1_dining = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_dining = tf.keras.layers.BatchNormalization()
    self.drop2_dining = tf.keras.layers.Dropout(dropout)
    #self.dense2_dining = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_dining = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
   
    # hall
    self.bn1_hall = tf.keras.layers.BatchNormalization()
    self.drop1_hall = tf.keras.layers.Dropout(dropout)
    self.dense1_hall = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_hall = tf.keras.layers.BatchNormalization()
    self.drop2_hall = tf.keras.layers.Dropout(dropout)
    #self.dense2_hall = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_hall = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
    
      # kitchen
    self.bn1_kitchen = tf.keras.layers.BatchNormalization()
    self.drop1_kitchen = tf.keras.layers.Dropout(dropout)
    self.dense1_kitchen = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_kitchen = tf.keras.layers.BatchNormalization()
    self.drop2_kitchen = tf.keras.layers.Dropout(dropout)
    #self.dense2_kitchen = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_kitchen = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))
  
      # livingroom
    self.bn1_living = tf.keras.layers.BatchNormalization()
    self.drop1_living = tf.keras.layers.Dropout(dropout)
    self.dense1_living = tf.keras.layers.Dense(nodes1, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.bn2_living = tf.keras.layers.BatchNormalization()
    self.drop2_living = tf.keras.layers.Dropout(dropout)
    #self.dense2_living = tf.keras.layers.Dense(nodes2, activation=tf.nn.relu, kernel_regularizer = keras.regularizers.l2(l2))
    self.dense2_living = tf.keras.layers.Dense(nodes2, kernel_regularizer = keras.regularizers.l2(l2))

    # final prediction
    self.bn_final = tf.keras.layers.BatchNormalization()
    self.drop_final = tf.keras.layers.Dropout(dropout)
    self.dense_final = tf.keras.layers.Dense(1, kernel_constraint =weight_constr())
    
  def call(self, inputs, training = None):
    
    # bathroom
    x = self.bn1_bath(inputs[0])
    x = self.drop1_bath(x)
    x = self.dense1_bath(x)
    x = self.bn2_bath(x)
    x = self.drop2_bath(x)
    bath_out = self.dense2_bath(x)
    
    
    # bedroom
    x = self.bn1_bed(inputs[1])
    x = self.drop1_bed(x)
    x = self.dense1_bed(x)
    x = self.bn2_bed(x)
    x = self.drop2_bed(x)
    bed_out = self.dense2_bed(x)
    
    # diningroom
    x = self.bn1_dining(inputs[2])
    x = self.drop1_dining(x)
    x = self.dense1_dining(x)
    x = self.bn2_dining(x)
    x = self.drop2_dining(x)
    dining_out = self.dense2_dining(x)
    
    # hallroom
    x = self.bn1_hall(inputs[3])
    x = self.drop1_hall(x)
    x = self.dense1_hall(x)
    x = self.bn2_hall(x)
    x = self.drop2_hall(x)
    hall_out = self.dense2_hall(x)
    
    # kitchen
    x = self.bn1_kitchen(inputs[4])
    x = self.drop1_kitchen(x)
    x = self.dense1_kitchen(x)
    x = self.bn2_kitchen(x)
    x = self.drop2_kitchen(x)
    kitchen_out = self.dense2_kitchen(x)
    
    # livingroom
    x = self.bn1_living(inputs[5])
    x = self.drop1_living(x)
    x = self.dense1_living(x)
    x = self.bn2_living(x)
    x = self.drop2_living(x)
    living_out = self.dense2_living(x)
    
    # join
    out = tf.keras.layers.concatenate([bath_out, bed_out, dining_out, hall_out, kitchen_out, living_out])#, others_out])
    out = self.bn_final(out)
    out = self.drop_final(out)
    return self.dense_final(out)

In [27]:
model_img = Img_model(dropout = 0.2, l2 = 0, nodes1 = 64, nodes2 = 1)
model_img.compile()
model_img.fit(train_dataset, epochs = 1)
model_img.load_weights("img_models/price/0/checkpoint"
)

2022-03-01 14:20:29.796579: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


22/22 [==============================] - 8s 288ms/step - loss: 0.0000e+00


2022-03-01 14:20:36.810053: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open img_models/price/0/checkpoint: Failed precondition: img_models/price/0/checkpoint; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


## Train Compound with reloaded weights (frozen)

In [57]:
# Load data
X_train, X_test, X_val, y_train, y_test, y_val = load_data_fold(fold=4, drop_id = False)


# set columns for tabnet
bin_col = [col for col in X_train if np.isin(X_train[col].unique(), [0, 1]).all()]
num_col = [col for col in X_train if ~np.isin(X_train[col].unique(), [0, 1]).all()]
col_names = bin_col + num_col
feature_columns = []
col_names.remove("id")
for col in col_names:
    feature_columns.append(tf.feature_column.numeric_column(col))
    
def transform(ds):
    # tabnet
    features = tf.unstack(ds["features"])
    features = dict(zip(col_names, features))

    # images
    bath = tf.unstack(ds["bath"])
    bed = tf.unstack(ds["bed"])
    dining = tf.unstack(ds["dining"])
    hall = tf.unstack(ds["hall"])
    kitchen = tf.unstack(ds["kitchen"])
    living = tf.unstack(ds["living"])
    
    prices = ds["price"]
    
    y = prices
    return (bath, bed, dining, hall, kitchen, living, features), y

In [58]:
# merge data for tabnet and images to get consistent ids
data_df_train = pd.merge(final_df, X_train, on="id", how="inner")
data_df_train.dropna(inplace = True)
print("TRAIN SHAPE: ", data_df_train.shape)

data_df_test = pd.merge(final_df, X_test, on="id", how="inner")
data_df_test.dropna(inplace = True)
print("TEST SHAPE: ", data_df_test.shape)

data_df_val = pd.merge(final_df, X_val, on="id", how="inner")
data_df_val.dropna(inplace = True)
print("VAL SHAPE: ", data_df_val.shape)

# extract consistent columns
X_train_tab = data_df_train.filter(col_names)
X_test_tab = data_df_test.filter(col_names)
X_val_tab = data_df_val.filter(col_names)

data_df_train.drop(col_names, axis = 1, inplace = True)
data_df_test.drop(col_names, axis = 1, inplace = True)
data_df_val.drop(col_names, axis = 1, inplace = True)

# extract consistent price
y_train = data_df_train.pop("log_price")
y_test = data_df_test.pop("log_price")
y_val = data_df_val.pop("log_price")

# drop id
X_train = data_df_train.drop("id", axis = 1)
X_test = data_df_test.drop("id", axis = 1)
X_val = data_df_val.drop("id", axis = 1)


TRAIN SHAPE:  (2710, 77)
TEST SHAPE:  (853, 77)
VAL SHAPE:  (706, 77)


In [59]:
train_size = int(X_train.shape[0] * 0.9)
batch_size = int(X_train.shape[0] * 0.1)

# X_train, X_test, y_train, y_test = train_test_split(features, prices, random_state = 123, test_size = 0.2)
# X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.2, random_state=1) # 0.25 x 0.8 = 0.2

data_train = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_train["bath"])),
                                                 "bed": np.squeeze(np.stack(X_train["bed"])),
                                                 "dining": np.squeeze(np.stack(X_train["dining"])),
                                                 "hall": np.squeeze(np.stack(X_train["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_train["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_train["living"])),
                                                 "features": X_train_tab,
                                                 "price": y_train})
data_train = data_train.cache()
data_train = data_train.shuffle(6000, seed = 13)
train_dataset = data_train.take(len(y_train))
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.batch(batch_size)

data_test = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_test["bath"])),
                                                 "bed": np.squeeze(np.stack(X_test["bed"])),
                                                 "dining": np.squeeze(np.stack(X_test["dining"])),
                                                 "hall": np.squeeze(np.stack(X_test["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_test["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_test["living"])),
                                                 "features": X_test_tab,
                                                 "price": y_test})
data_test = data_test.cache()
test_dataset = data_test.take(len(y_test))
test_dataset = test_dataset.map(transform)
test_dataset = test_dataset.batch(batch_size)

data_val = tf.data.Dataset.from_tensor_slices({"bath": np.squeeze(np.stack(X_val["bath"])),
                                                 "bed": np.squeeze(np.stack(X_val["bed"])),
                                                 "dining": np.squeeze(np.stack(X_val["dining"])),
                                                 "hall": np.squeeze(np.stack(X_val["hall"])),
                                                 "kitchen": np.squeeze(np.stack(X_val["kitchen"])),
                                                 "living": np.squeeze(np.stack(X_val["living"])),
                                                 "features": X_val_tab,
                                                 "price": y_val})
data_val = data_val.cache()
val_dataset = data_val.take(len(y_val))
val_dataset = val_dataset.map(transform)
val_dataset = val_dataset.batch(batch_size)

In [61]:
tf.random.set_seed(123)
model = Compound_model(feature_columns = feature_columns,
                                output_dim=120, feature_dim=125, num_groups=1,
                                num_decision_steps=2, relaxation_factor=1.5,
                                sparsity_coefficient=1e-05, dropout= 0.2, nodes1 = 64, nodes2 = 1, l2 = 0)

lr = tf.keras.optimizers.schedules.ExponentialDecay(0.01, decay_steps=100, decay_rate=0.95, staircase=False)
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss='mse' , metrics=[R_squared,"mae"])

model.fit(train_dataset, validation_data = val_dataset, epochs=1, verbose=1)

# set layer weights to pretrained weights
model.layers[1:-4] = model_img.layers
model.layers[0] = model_tn.layers[0]

# freeze layer weights
for layer in model.layers[:-4]:
    layer.trainable = False
    
logdir = "logs/price/compound/final_frozen"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = logdir+"/checkpoint.ckpt", monitor='val_R_squared', save_best_only=True, mode='max')
    
model.fit(train_dataset, validation_data = val_dataset, epochs=300, verbose=1, callbacks = [checkpoint_callback])


[TabNet]: 5 features will be used for decision steps.


2022-03-01 18:21:37.976133: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - ETA: 0s - loss: 21.5971 - R_squared: -52.1608 - mae: 4.5296

2022-03-01 18:22:16.853368: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 76s 4s/step - loss: 21.5971 - R_squared: -52.1608 - mae: 4.5296 - val_loss: 15.7903 - val_R_squared: -36.3999 - val_mae: 3.7730
Epoch 1/300
10/10 [==============================] - 29s 3s/step - loss: 19.3269 - R_squared: -46.4065 - mae: 4.2607 - val_loss: 15.7703 - val_R_squared: -35.9743 - val_mae: 3.8126


...

INFO:tensorflow:Assets written to: logs/price/compound/final_frozen/checkpoint.ckpt/assets


Epoch 114/300
10/10 [==============================] - 10s 1s/step - loss: 0.5443 - R_squared: -0.3290 - mae: 0.4866 - val_loss: 0.2957 - val_R_squared: 0.3466 - val_mae: 0.4135
Epoch 115/300
10/10 [==============================] - 11s 1s/step - loss: 0.6293 - R_squared: -0.5509 - mae: 0.5261 - val_loss: 0.2820 - val_R_squared: 0.3706 - val_mae: 0.4017
Epoch 116/300
10/10 [==============================] - 12s 1s/step - loss: 0.5930 - R_squared: -0.4588 - mae: 0.5083 - val_loss: 0.2737 - val_R_squared: 0.3902 - val_mae: 0.3925
Epoch 117/300
10/10 [==============================] - 11s 1s/step - loss: 0.6739 - R_squared: -0.6528 - mae: 0.5415 - val_loss: 0.2859 - val_R_squared: 0.3608 - val_mae: 0.4073
Epoch 118/300
10/10 [==============================] - 12s 1s/step - loss: 0.5670 - R_squared: -0.3948 - mae: 0.4951 - val_loss: 0.2875 - val_R_squared: 0.3538 - val_mae: 0.4098
Epoch 119/300
10/10 [==============================] - 11s 1s/step - loss: 0.6016 - R_squared: -0.4708 - mae: 

INFO:tensorflow:Assets written to: logs/price/compound/final_frozen/checkpoint.ckpt/assets


INFO:tensorflow:Assets written to: logs/price/compound/final_frozen/checkpoint.ckpt/assets


Epoch 165/300
10/10 [==============================] - 11s 1s/step - loss: 0.3965 - R_squared: 0.0324 - mae: 0.4080 - val_loss: 0.2646 - val_R_squared: 0.4180 - val_mae: 0.3924
Epoch 166/300
10/10 [==============================] - 13s 1s/step - loss: 0.3730 - R_squared: 0.0842 - mae: 0.3953 - val_loss: 0.2560 - val_R_squared: 0.4404 - val_mae: 0.3848
Epoch 167/300
10/10 [==============================] - 13s 1s/step - loss: 0.3466 - R_squared: 0.1490 - mae: 0.3780 - val_loss: 0.2643 - val_R_squared: 0.4229 - val_mae: 0.3923
Epoch 168/300
10/10 [==============================] - 13s 1s/step - loss: 0.3228 - R_squared: 0.2104 - mae: 0.3607 - val_loss: 0.2649 - val_R_squared: 0.4199 - val_mae: 0.3922
Epoch 169/300
10/10 [==============================] - 11s 1s/step - loss: 0.3336 - R_squared: 0.1816 - mae: 0.3654 - val_loss: 0.2657 - val_R_squared: 0.4136 - val_mae: 0.3945
Epoch 170/300
10/10 [==============================] - 11s 1s/step - loss: 0.3583 - R_squared: 0.1223 - mae: 0.3782

In [67]:
model.load_weights("logs/price/compound/final_frozen/checkpoint.ckpt")
model.layers[-1].get_weights()

2022-03-02 08:03:36.985329: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open logs/price/compound/final_frozen/checkpoint.ckpt: Failed precondition: logs/price/compound/final_frozen/checkpoint.ckpt; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


[array([[-0.       ],
        [ 1.0000001]], dtype=float32),
 array([3.5302231], dtype=float32)]

In [71]:
# freeze layer weights
for layer in model.layers[:-4]:
    layer.trainable = False
model.summary()

Model: "compound_model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tab_net_7 (TabNet)           multiple                  177121    
_________________________________________________________________
batch_normalization_97 (Batc multiple                  8192      
_________________________________________________________________
dropout_97 (Dropout)         multiple                  0         
_________________________________________________________________
dense_103 (Dense)            multiple                  131136    
_________________________________________________________________
batch_normalization_98 (Batc multiple                  256       
_________________________________________________________________
dropout_98 (Dropout)         multiple                  0         
_________________________________________________________________
dense_104 (Dense)            multiple             

In [63]:
model.evaluate(train_dataset)

10/10 [==============================] - 6s 518ms/step - loss: 0.1835 - R_squared: 0.5536 - mae: 0.3277


[0.1834837645292282, 0.5535845756530762, 0.3277072310447693]

In [64]:
model.evaluate(val_dataset)

3/3 [==============================] - 2s 391ms/step - loss: 0.2446 - R_squared: 0.4647 - mae: 0.3720


[0.24461446702480316, 0.4647457003593445, 0.3720126748085022]

In [65]:
model.evaluate(test_dataset)

4/4 [==============================] - 3s 507ms/step - loss: 0.2372 - R_squared: 0.4423 - mae: 0.3735


[0.23721563816070557, 0.44234317541122437, 0.3734670579433441]

### Unfreeze weights

In [66]:
for layer in model.layers[:-4]:
    layer.trainable = True
    
logdir = "logs/price/compound/final_unfrozen"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath = logdir+"/checkpoint.ckpt", monitor='val_R_squared', save_best_only=True, mode='max')

lr = 0.0001
optimizer = tf.keras.optimizers.Adam(lr)
model.compile(optimizer, loss='mse' , metrics=[R_squared,"mae"])
model.load_weights("logs/price/compound/final_frozen/checkpoint.ckpt")

model.fit(train_dataset, validation_data = val_dataset, epochs=300, verbose=1, callbacks = [checkpoint_callback])

2022-03-01 22:06:19.178205: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open logs/price/compound/final_frozen/checkpoint.ckpt: Failed precondition: logs/price/compound/final_frozen/checkpoint.ckpt; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


Epoch 1/300


2022-03-01 22:06:23.512580: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - ETA: 0s - loss: 0.3618 - R_squared: 0.1186 - mae: 0.3897

2022-03-01 22:06:30.903306: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 12s 741ms/step - loss: 0.3618 - R_squared: 0.1186 - mae: 0.3897 - val_loss: 0.2460 - val_R_squared: 0.4618 - val_mae: 0.3731


INFO:tensorflow:Assets written to: logs/price/compound/final_unfrozen/checkpoint.ckpt/assets


INFO:tensorflow:Assets written to: logs/price/compound/final_unfrozen/checkpoint.ckpt/assets


Epoch 2/300
10/10 [==============================] - 9s 868ms/step - loss: 0.3858 - R_squared: 0.0519 - mae: 0.4006 - val_loss: 0.2487 - val_R_squared: 0.4561 - val_mae: 0.3752
Epoch 3/300
10/10 [==============================] - 8s 813ms/step - loss: 0.3785 - R_squared: 0.0771 - mae: 0.3930 - val_loss: 0.2517 - val_R_squared: 0.4494 - val_mae: 0.3775
Epoch 4/300
10/10 [==============================] - 10s 951ms/step - loss: 0.3614 - R_squared: 0.1149 - mae: 0.3828 - val_loss: 0.2552 - val_R_squared: 0.4419 - val_mae: 0.3802
Epoch 5/300
10/10 [==============================] - 8s 815ms/step - loss: 0.3467 - R_squared: 0.1490 - mae: 0.3768 - val_loss: 0.2582 - val_R_squared: 0.4354 - val_mae: 0.3832
Epoch 6/300
10/10 [==============================] - 8s 806ms/step - loss: 0.3741 - R_squared: 0.0848 - mae: 0.3910 - val_loss: 0.2608 - val_R_squared: 0.4297 - val_mae: 0.3861
Epoch 7/300
10/10 [==============================] - 8s 797ms/step - loss: 0.3734 - R_squared: 0.0807 - mae: 0.385

In [73]:
model.load_weights("logs/price/compound/final_unfrozen/checkpoint.ckpt")
model.layers[-1].get_weights()

2022-03-02 08:14:16.110196: W tensorflow/core/util/tensor_slice_reader.cc:95] Could not open logs/price/compound/final_unfrozen/checkpoint.ckpt: Failed precondition: logs/price/compound/final_unfrozen/checkpoint.ckpt; Is a directory: perhaps your file is in a different file format and you need to use a different restore operator?


[array([[-0.],
        [ 1.]], dtype=float32),
 array([3.530381], dtype=float32)]